In [3]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

In [4]:
stopwords=['ଦେଇଛନ୍ତି', 'ଲେଖାଏଁ', 'ଜଣେ', 'ଏହା', 'ତେଣୁ', 'ମିଳିଥାଏ', 'ପାଇଁ', 'ନେଉଛନ୍ତି', 'ଯୋଗୁଁ', 'ଏପର୍ଯ୍ୟନ୍ତ', 'ଏଭଳି', 'କରୁଛି', 'ଓ', 'ଯାଏଁ','ହୋଇଛନ୍ତି', 'କି', 'କରାଯିବା', 'ପରେ', 'ଏହି', 'ଏବଂ', 'ଜଣ', 'ଥିବା','ହୋଇଥିଲା', 'ତେବେ', 'ଆଜି', 'ଜଣଙ୍କ', 'ଏଥି', 'ଗତ', 'ହୋଇଥିଲେ',
       'ଦିନ ଧରି', 'ହେଉଥିବା', 'ଦିନ ତଳେ', 'ଯୋଗେ', 'ବୋଲି', 'ଜଣାପଡ଼ିଛି',
       'ଦ୍ବାରା', 'କରି', 'ଯାଇ', 'ଏ ନେଇ', 'ଚାଲୁ', 'ରହିଛି', 'ତାରିଖ',
       'ମିଳିଛି', 'ବର୍ଷୀୟ', 'ଦିନ ତଳେ', 'ସହ', 'ଆସିଛନ୍ତି', 'ମଧ୍ୟ', 'କେଉଁ',
       'ହୋଇଯାଇଛି', 'ନେଇ ଯାଇଛି', 'କାମ', 'କରିଛି', 'ହେବା', 'ପରେ', 'ଏବେ',
       'ହୋଇଛି', 'ତେବେ', 'ରଖି', 'ନିଆଯାଇଥିବା', 'ପର୍ଯ୍ୟନ୍ତ', 'କରିଛନ୍ତି',
       'ଉପରେ', 'ଦେଲେ', 'ଥର', 'ଏବଂ', 'ଆଉ']
alphabets=['ଅ','ଆ','ଇ','ଈ','ଉ','ଊ','ଋ','ଌ','ଏ','ଐ','ଓ','ଔ','କ','ଖ','ଗ','ଘ','ଙ','ଚ','ଛ','ଜ','ଝ','ଞ','ଟ','ଠ','ଡ','ଢ','ଣ',
 'ତ','ଥ','ଦ','ଧ','ନ','ପ','ଫ','ବ','ଭ','ମ','ଯ','ର','ଲ','ଳ','ଵ','ଶ','ଷ','ସ','ହ','ଡ଼','ଢ଼','ୟ','ୠ','ୡ','ୱ']
matra=['ଁ','ଂ','ଃ','଼','ା','ି','ୀ','ୁ','ୂ','ୃ','ୄ','େ','ୈ','ୋ','ୌ','୍','ୢ','ୣ']

In [5]:
data=pd.read_csv('../odia.csv')

In [6]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())


In [7]:
with open('pickles/df.pkl','rb') as file:
    DF=pickle.load(file)
    file.close()
    
with open('pickles/file_idx.pkl','rb') as file:
    file_idx=pickle.load(file)
    file.close()
    
with open('pickles/doc_words.pkl','rb') as file:
    doc_words=pickle.load(file)
    file.close()
    
with open('pickles/doc_norm.pkl','rb') as file:
    doc_norm=pickle.load(file)
    file.close()

In [8]:
def tfidf(query,counter=10):
    text = remove_special_characters(query)
    words = text.split()
    words=[word for word in words if word not in stopwords]
    words=[word for word in words if word in DF.keys()]

    q=[]
    q_norm=0
    for w in words:
        tf_idf=(words.count(w)*math.log(len(file_idx)/DF[w]))
        q.append(tf_idf)
        q_norm+=tf_idf**2
    q_norm=math.sqrt(q_norm)

    q=np.array(q)/q_norm

    score={}

    for i in range(len(file_idx)):
        doc_v=[]
        for w in words:
            tf_idf=(doc_words[i].count(w)*math.log(len(file_idx)/DF[w]))
            doc_v.append(tf_idf)
        doc_v=np.array(doc_v)/doc_norm[i]
        score[i]=np.dot(q,doc_v)

    score=sorted(score.items(),key=lambda x:x[1],reverse=True)
    verse_index=[]
    for i in score:
        if counter== 0:
            break
        verse_index.append(i[0])
        counter-=1
    return verse_index
    

In [11]:
verse_index=tfidf('ସବୁଠାରୁ ଉପଯୁକ୍ତ କାର୍ଯ୍ୟ ପଦ୍ଧତି ବିଷୟରେ ଅନୁମାନ କରିବାକୁ ଯାଇ, ଆମେ ସେମାନଙ୍କର ପରିଣାମ ସମ୍ବନ୍ଧୀୟ ବିଭିନ୍ନ ବିକଳ୍ପ ଉପରେ ବିଚାର କରିଥାଉ । ଅର୍ଜୁନ ବିଚାର କରୁଛନ୍ତି, ଯୁଦ୍ଧରେ କୌରବମାନଙ୍କୁ ପରାଜିତ କରିବା ଉଚିତ ନା କୌରବଙ୍କ ଦ୍ୱାରା ପରାଜିତ ହେବା ଉଚିତ? ଉଭୟ ବିକଳ୍ପ ପରାଜୟ ପରି ହିଁ ପ୍ରତୀତ ହେଉଛି, କାରଣ କୌରବମାନଙ୍କୁ ହତ୍ୟା କରି ସେ ଯଦି ଯୁଦ୍ଧରେ ଜୟ ଲାଭ କରନ୍ତି, ତେବେ ତାଙ୍କର ବଞ୍ôଚବାର ଇଚ୍ଛା ରହିବ ନାହିଁ ।କିନ୍ତୁ ବାସ୍ତବରେ ଭୀଷ୍ମ, ଦ୍ରୋଣ, କୃପାଚାର୍ଯ୍ୟ ଇତ୍ୟାଦି ବିଧର୍ମୀ କୌରବମାନଙ୍କର ପକ୍ଷ ନେଇ ନିକୃଷ୍ଟ ଆଚରଣ ପ୍ରଦର୍ଶନ କରିଛନ୍ତି । ଅର୍ଥକାମ ଶବ୍ଦଟି ଏଠାରେ ସେମାନଙ୍କ ପାଇଁ ବ୍ୟବହାର କରାଯାଇଛି, ଯାହାର ଅର୍ଥ “ବୈଭବ ଏବଂ ପଦବୀ ପ୍ରତି ଆସକ୍ତ ରହି” ସେମାନେ ଦୁଷ୍ଟ ଦୁର୍ଯ୍ୟୋଧନଙ୍କର ପକ୍ଷ ସ୍ୱୀକାର କରିଥିଲେ । ତେଣୁ ଯୁଦ୍ଧରେ ସେମାନଙ୍କୁ ହତ୍ୟା କରିବା ଏକ ସ୍ୱାଭାବିକ ପରିଣତି ଥିଲା । ପ୍ରକୃତରେ ଯୁଦ୍ଧ ଶେଷ ହେବା ପରେ, ଭୀଷ୍ମ ନିଜେ ସ୍ୱୀକାର କରିଥିଲେ ଯେ ଜଣେ ଶିକ୍ଷକ ଯଦି ନୀଚ କର୍ମ କରନ୍ତି, ତେବେ\xa0 ତାଙ୍କୁ ତ୍ୟାଗ କରାଯିବା ଉଚିତ୍ ।ଏଠାରେ ଭୀଷ୍ମଙ୍କ ବିଷୟରେ ସ୍ୱତନ୍ତ୍ର ବିଚାର ଆବଶ୍ୟକ । ଶ୍ରୀମଦ୍ ଭାଗବଦମ୍\xa0 (ଶ୍ଲୋକ ୯.୨୨.୧୯) ଅନୁସାରେ, ସେ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ପରମ ଭକ୍ତ ଥିଲେ । ସେ ଇନ୍ଦ୍ରିୟଜୟୀ ଥିଲେ ଏବଂ ନୈତିକତା ଓ ଉଦାରତାର ମୂର୍ତ୍ତିମାନ ସ୍ୱରୂପ ଥିଲେ । ସେ ଚିରନ୍ତନ ସତ୍ୟର ଜ୍ଞାତା ଏବଂ ଜୀବନରେ ସତ୍ୟବାଦୀ ହେବାକୁ ପ୍ରତିଜ୍ଞାବଦ୍ଧ ଥିଲେ । ସେ ଇଚ୍ଛା କଲେ ହିଁ ତାଙ୍କର ମୃତ୍ୟୁ ହୋଇପାରେ । ବିଭିନ୍ନ ଦୃଷ୍ଟିକୋଣରୁ, ଭାଗବତ ମହାପୁରାଣରେ ବର୍ଣ୍ଣିତ ବାର ଜଣ ଶ୍ରେଷ୍ଠ ବ୍ୟକ୍ତି ବା ମହାଜନଙ୍କ ମଧ୍ୟରେ ତାଙ୍କର ଗଣନା କରାଯାଇଥାଏ ।\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 ସ୍ୱୟମ୍ଭୂର୍ ନାରଦଃ ଶମ୍ଭୁଃ କୁମାରଃ କପିଳୋ ମନୁଃ\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 ପ୍ରହ୍ଲାଦୋ ଜନକୋ ଭୀଷ୍ମୋ ବଳିର୍ ବୈୟାଶକିର୍ ବୟମ୍ । (୬.୩.୨୦)ଏହି ବାର ଜଣ ଧାର୍ମିକ ସିଦ୍ଧାନ୍ତର ମହାନ ଜ୍ଞାତା ଅଟନ୍ତି - ପ୍ରଥମ ପୁରୁଷ ବ୍ରହ୍ମା, ମହର୍ଷି ନାରଦ, ଭଗବାନ ଶିବ, ଚାରି କୁମାର, ଭଗବାନ କପିଳ (ଦେବହୁତିଙ୍କ ପୁତ୍ର), ସ୍ୱୟମ୍ଭୁବ ମନୁ, ପ୍ରହ୍ଲାଦ ମହାରାଜ, ମହାରାଜ ଜନକ, ପିତାମହ ଭୀଷ୍ମ, ମହାରାଜ ବଳି, ଶୁକଦେବ ମୁନି ଏବଂ ବେଦବ୍ୟାସ ।ତେଣୁ ଭୀଷ୍ମ ଜଣେ ଧର୍ମାତ୍ମା ଥିଲେ, ତାଙ୍କର ଆଚରଣ ଧର୍ମନୀତିର ବିରୁଦ୍ଧ କେବେ ବି ହୋଇ ନ ପାରେ । ତେଣୁ ତାଙ୍କର ବଳିଷ୍ଠ ଚରିତ୍ର ପାର୍ôଥବ ଯୁକ୍ତିର ଉର୍ଦ୍ଧ୍ୱରେ ଥିଲା । ଯଦିଓ ସେ କୌରବଙ୍କ ପକ୍ଷରୁ ଯୁଦ୍ଧ କରୁଥିଲେ, ତଥାପି ଯୁଦ୍ଧ ପୁର୍ବରୁ ସେ ଯୁଧିଷ୍ଠିରଙ୍କୁ କହିଥିଲେ, “ମୁଁ ଅଧର୍ମ ପକ୍ଷରୁ ଯୁଦ୍ଧ କରିବାକୁ ବାଧ୍ୟ, କିନ୍ତୁ ମୁଁ ତୁମକୁ ବିଜୟୀ ହେବାର ବର ପ୍ରଦାନ କରୁଅଛି ।” ଭୀଷ୍ମ ଜାଣିଥିଲେ ଯେ ଧାର୍ମିକ ପାଣ୍ଡବମାନେ, ଯାହା ସହିତ ସ୍ୱୟଂ ପରଂବ୍ରହ୍ମ ଶ୍ରୀକୃଷ୍ଣ ଅଛନ୍ତି, କେବେ ବି ପରାସ୍ତ ହେବେ ନାହିଁ । ଅଧର୍ମର ପକ୍ଷ ନେଇ ସେ ସୂଚାଇ ଦେଲେ ଯେ ସ୍ୱର୍ଗ ଓ ମର୍ତ୍ତ୍ୟର ସର୍ବଶେଷ୍ଠ ଶକ୍ତି ମଧ୍ୟ ଏହି ଧର୍ମ ଯୁଦ୍ଧରେ ଅଧର୍ମକୁ ବିଜୟୀ କରାଇ ପାରିଲା ନାହିଁ । ଏହିପରି ସର୍ବଶ୍ରେଷ୍ଠ ତ୍ୟାଗ ଭାବରେ ନିଜ ଜୀବନକୁ ଉତ୍ସର୍ଗ କରି ସେ ଶ୍ରୀକୃଷ୍ଣଙ୍କର ଦିବ୍ୟ ଲୀଳାରେ ସହାୟତା କରିଥିଲେ ।ଭୀଷ୍ମଙ୍କର ତାଙ୍କ ପ୍ରତି ଥିବା ଗଭୀର ଭକ୍ତି ବିଷୟରେ ଶ୍ରୀକୃଷ୍ଣ ଜାଣିଥିଲେ, ଯଦିଓ ସେ କୌରବଙ୍କ ସପକ୍ଷରେ ଯୁଦ୍ଧ କରୁଥିଲେ । ସେଥିପାଇଁ ଭୀଷ୍ମଙ୍କର ପ୍ରତିଜ୍ଞା ରକ୍ଷା କରିବାକୁ ଯାଇ ସେ ନିଜର ପ୍ରତିଜ୍ଞା ଭଙ୍ଗ କରିଥିଲେ । ଯୁଦ୍ଧରେ ଦିନେ ଭୀଷ୍ମ ପ୍ରତିଜ୍ଞା କଲେ ଯେ ପରଦିନ ସୂର୍ଯ୍ୟାସ୍ତ ପୂର୍ବରୁ ସେ ପାଣ୍ଡବମାନଙ୍କର ଅଗ୍ରଣୀ ଯୋଦ୍ଧା ଅର୍ଜୁନଙ୍କୁ ବଧ କରିବେ ଅଥବା ଅର୍ଜୁନଙ୍କୁ ରକ୍ଷା କରିବାକୁ ଯାଇ ଶ୍ରୀକୃଷ୍ଣ ମହାଭାରତ ଯୁଦ୍ଧରେ ଅସ୍ତ୍ର ଧାରଣ ନ କରିବାର ଯେଉଁ ପ୍ରତିଜ୍ଞା କରିଥିଲେ, ତାହାକୁ ଭଙ୍ଗ କରିବେ । ଭୀଷ୍ମଙ୍କର ସେହି ପ୍ରତିଜ୍ଞାର ବର୍ଣ୍ଣନ କବିମାନେ ଏହିପରି କରିଛନ୍ତି -\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 ଆଜୁ ଜୋ ହରି ହିଁ ନ ଶସ୍ତ୍ର ଗହାଊଁ,\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 ତଉ ଲାଜହୁଁ ଗଙ୍ଗା ଜନନୀ କୋ ଶାନ୍ତନୁ ସୁତ ନ କହାଊଁ । (ସନ୍ଥ ତୁଳସୀ ଦାସ)ଯଦି ମୁଁ ଭଗବାନ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ହସ୍ତରେ ଅସ୍ତ୍ର ଧାରଣ ନ କରାଏ, ତେବେ ମୁଁ ନିଜ ମାତା ଗଙ୍ଗାଙ୍କୁ ଲଜ୍ଜିତ କରାଇବି ଏବଂ ମୁଁ ରାଜା ଶାନ୍ତନୁଙ୍କ ପୁତ୍ର ବୋଲାଇବି ନାହିଁ । ଭୀଷ୍ମ ଏପରି ବୀରତ୍ୱର ସହିତ ଯୁଦ୍ଧ କଲେ ଯେ ଅର୍ଜୁନଙ୍କ ରଥ ଛିନ୍ନଛତ୍ର ହୋଇଗଲା ଏବଂ ସେ ଭୂମି ଉପରେ ଅସହାୟ ଭାବେ ଛିଡ଼ା ହୋଇଗଲେ । ସେହି ମୁହୂର୍ତ୍ତରେ ଭୀଷ୍ମଙ୍କୁ ବାଧା ଦେବା ପାଇଁ, ଶ୍ରୀକୃଷ୍ଣ ରଥର ଏକ ଚକକୁ ଉଠାଇ ତାଙ୍କ ସମ୍ମୁଖକୁ ଆସିଗଲେ । ରଥର ଚକକୁ ଅସ୍ତ୍ର ଭାବରେ ଭଗବାନ ହାତରେ ଧାରଣ କରିଥିବା ଦେଖି, ଭୀଷ୍ମ ଅଟ୍ଟହାସ୍ୟ କରି ଉଠିଲେ । ସେ ବୁଝିପାରିଲେ ଯେ ଭକ୍ତ ବତ୍ସଳ ଭଗବାନ, ନିଜ ଭକ୍ତର ପ୍ରତିଜ୍ଞା ରକ୍ଷା କରିବା ପାଇଁ ନିଜ ପ୍ରତିଜ୍ଞା ଭଙ୍ଗ କଲେ ।ବାସ୍ତବରେ, ଭଗବାନ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ପ୍ରତି ଭୀଷ୍ମଙ୍କର ଭକ୍ତି ଅତ୍ୟନ୍ତ ରସିକ(ମାଧୁର୍ଯ୍ୟ) ଭାବଯୁକ୍ତ ଥିଲା । ସେ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ବୃନ୍ଦାବନ ଲୀଳାର ଧ୍ୟାନ କରୁଥିଲେ । ସନ୍ଧ୍ୟା ସମୟରେ ଶ୍ରୀକୃଷ୍ଣ ଯେତେବେଳେ ଗୋଚାରଣ ସାରି ଗ୍ରାମକୁ ଫେରନ୍ତି, ଗାଈମାନଙ୍କ ଖୁରାରୁ ଉଡ଼ୁଥିବା ଧୂଳିରେ ତାଙ୍କ ସୁନ୍ଦର ମୁଖମଣ୍ଡଳ ଆବୃତ ହୋଇ ତାଙ୍କର ସୌନ୍ଦର୍ଯ୍ୟ ଓ ମାଧୁର୍ଯ୍ୟକୁ ଆହୁରି ବଢ଼ାଇ ଦିଏ । ମହାଭାରତ ଯୁଦ୍ଧରେ ଘୋଡ଼ାମାନଙ୍କର ଟାପୁରୁ ଉଡ଼ୁଥିବା ଧୂଳି ମଧ୍ୟ ଶ୍ରୀକୃଷ୍ଣଙ୍କର ସୌନ୍ଦର୍ଯ୍ୟକୁ ବୃଦ୍ଧି କରୁଥିଲା ଏବଂ ସେ ଭଗବାନଙ୍କର ସେହି ରୂପର ଦର୍ଶନ କରି ବିଭୋର ହୋଇ ଯାଉଥିଲେ ।ଜୀବନର ଅନ୍ତିମ ପର୍ଯ୍ୟାୟରେ, ସେ ଛ’ମାସ ଶରଶଯ୍ୟାରେ ଶୟନ କରି ଭଗବାନଙ୍କର ସେହି ରୂପର ଧ୍ୟାନ କରୁଥିଲେ ଏବଂ ପ୍ରାର୍ଥନା କରୁଥିଲେ:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 ଯୁଧି ତୁରଗ-ରଜୋ-ବିଧୂମ୍ର-ବିଷ୍ୱକ୍\u200c-କଚ-ଲୁଲିତ-ଶ୍ରମବାର୍ଯ୍ୟଲଙ୍\u200cକୃତାସ୍ୟେ\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 ମମ ନିଶିତ-ଶରୈର୍ବିଭିଦ୍ୟମାନ ତ୍ୱଚି ବିଲସତ୍\u200c-କବଚେଽସ୍ତୁ କୃଷ୍ଣ ଆତ୍ମା । (ଭାଗବତମ୍ : ୧.୯.୩୪)“ଯୁଦ୍ଧ କ୍ଷେତ୍ରରେ, ଭଗବାନ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ମସ୍ତକର ତରଙ୍ଗାୟିତ କେଶରାଶି ଘୋଡ଼ାଙ୍କ ଖୁରାରୁ ଉଡ଼ୁଥିବା ଶୁଭ୍ର ଧୂଳି ଦ୍ୱାର ଧୂସରିତ ଦେଖା ଯାଉଥିଲା ଏବଂ ରଥ ଚାଳନା ଜନିତ ପରିଶ୍ରମ ଦ୍ୱାରା ତାଙ୍କ ମୁଖମଣ୍ଡଳ ସ୍ୱେଦ ବିନ୍ଦୁରେ ପରିପୂର୍ଣ୍ଣ ହୋଇଥିଲା । ଏହି ସବୁ ଅଳଙ୍କାର ସଦୃଶ ମୋ ପ୍ରଭୁଙ୍କର ସୌନ୍ଦର୍ଯ୍ୟ ବୃଦ୍ଧି କରୁଥିଲା ଏବଂ ମୋର ଶାଣୀତ ତୀର ଦ୍ୱାରା ସୃଷ୍ଟି ହୋଇଥିବା କ୍ଷତ ତାଙ୍କର ରୂପ ସଜ୍ଜାକୁ ଆହୁରି ବୃଦ୍ଧି କରୁଥିଲା । ମୋର ମନ ସେହି ଶ୍ରୀକୃଷ୍ଣଙ୍କର ଧ୍ୟାନ କରୁ ।”ତାଙ୍କ ପ୍ରେମଯୁକ୍ତ ଭକ୍ତିର ଫଳ ସ୍ୱରୂପ, ସେ ଶରଶଯ୍ୟାରେ ଶୟନ କରିଥିବା ସମୟରେ ଭଗବାନ ଶ୍ରୀକୃଷ୍ଣ ତାଙ୍କୁ ସାକ୍ଷାତ କରିବାକୁ ଆସିଥିଲେ ଏବଂ ନିଜ ସମ୍ମୁଖରେ ଭଗବାନଙ୍କର ଦର୍ଶନ କରି, ଶ୍ରେଷ୍ଠ ମହାଜନ ଭୀଷ୍ମ ନିଜ ଇଚ୍ଛାରେ ଶରୀର ତ୍ୟାଗ କଲେ ।')
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
45,2,6,न चैतद्विद्मः कतरन्नो गरीयो\n\nयद्वा जयेम यदि ...,ସବୁଠାରୁ ଉପଯୁକ୍ତ କାର୍ଯ୍ୟ ପଦ୍ଧତି ବିଷୟରେ ଅନୁମାନ କ...
432,11,26,अमी च त्वां धृतराष्ट्रस्य पुत्राः\n\nसर्वे सहै...,ଭଗବାନଙ୍କ କେଉଁ ଦାନ୍ତ ବିଷୟରେ ଅର୍ଜୁନ କହୁଛନ୍ତି? ସେ...
433,11,27,वक्त्राणि ते त्वरमाणा विशन्ति\n\nदंष्ट्राकराला...,ଭଗବାନଙ୍କ କେଉଁ ଦାନ୍ତ ବିଷୟରେ ଅର୍ଜୁନ କହୁଛନ୍ତି? ସେ...
361,9,31,क्षिप्रं भवति धर्मात्मा शश्वच्छान्तिं निगच्छति...,"କେବଳ ଉଚିତ୍ ସଂକଳ୍ପ କଲେ, ଜଣେ ଭକ୍ତକୁ ଧର୍ମାତ୍ମା କୁ..."
43,2,4,अर्जुन उवाच\n\nकथं भीष्ममहं संख्ये द्रोणं च मध...,କର୍ମ କରିବା ପାଇଁ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ଆହ୍ୱାନର ପ୍ରତିକ୍ରି...
9,1,12,तस्य संजनयन्हर्षं कुरुवृद्धः पितामहः।\n\nसिंहन...,ପିତାମହ ଭୀଷ୍ମ ତାଙ୍କ ପୌପୁତ୍ରଙ୍କର ହୃଦୟର ଭୟ ବୁଝିପା...
7,1,10,अपर्याप्तं तदस्माकं बलं भीष्माभिरक्षितम्।\n\nप...,"ଦୁର୍ଯ୍ୟୋଧନଙ୍କର ଆତ୍ମଗରିମା ଗାନ, ଜଣେ ବୃଥା ବଡ଼ିମା ..."
74,2,35,भयाद्रणादुपरतं मंस्यन्ते त्वां महारथाः।\n\nयेष...,ଅର୍ଜୁନ ଜଣେ ବୀର ଯୋଦ୍ଧା ଭାବରେ ଖ୍ୟାତ ଥିଲେ ଓ କୌରବ ...
440,11,34,द्रोणं च भीष्मं च जयद्रथं च\n\nकर्णं तथाऽन्यान...,କୌରବ ପକ୍ଷର ଅନେକ ମହାରଥୀ ଏହା ପୂର୍ବରୁ ଯୁଦ୍ଧରେ ଅପର...
29,1,34,आचार्याः पितरः पुत्रास्तथैव च पितामहाः।\n\nमात...,ଦ୍ରୋଣାଚାର୍ଯ୍ୟ ଏବଂ କୃପାଚାର୍ଯ୍ୟ ଅର୍ଜୁନଙ୍କ ଗୁରୁ ଥ...
